# Setup

In [ ]:
# Load the relevant environment variables
%load_ext dotenv
%dotenv -v ../../.env

# activate autoreloading of modules
%load_ext autoreload
%autoreload 2

# import the db_module for database access
import sys
import os
from pathlib import Path

db_module = Path(os.path.dirname(os.path.realpath("__file__"))).parent / 'src'
sys.path.append(str(db_module))

# Query the database using SQLAlchemy

In [ ]:
from models.burningglass import BurningGlassPosting
from models.professions import OnetProfession
from models.textkernel import TextkernelPosting
from database.sql_session import session_scope
from database.sql import engine
import pandas as pd

# create a database session
with session_scope() as session:
    # query 100 textkernel postings from 2021
    tk_postings = pd.read_sql_query(
        sql = session.query(TextkernelPosting)\
            .filter(TextkernelPosting.published_at >= '20210101', TextkernelPosting.published_at < '20210131')\
            .limit(100)\
            .statement,
        con = engine
    )

    # query 100 burningglass postings from 2021
    bg_postings = pd.read_sql_query(
        sql = session.query(BurningGlassPosting)\
            .filter(BurningGlassPosting.year_grab_date == 2021)\
            .filter(BurningGlassPosting.month_grab_date == 1)\
            .filter(BurningGlassPosting.day_grab_date >= 1)\
            .filter(BurningGlassPosting.day_grab_date <= 31)\
            .limit(100)\
            .statement,
        con = engine
    )


# Show the results

In [ ]:
# Lets see textkernel postings
tk_postings

In [ ]:
# Lets see burningglass postings
bg_postings

# Get the full text data

In [ ]:
from utils.sftp_controller import Controller

sftp_controller = Controller()

id = tk_postings['id'][0]
published_at = tk_postings['published_at'][0]

obj = sftp_controller.get_file(id, published_at)

obj
    